In [90]:
using GeometryTypes
const gt = GeometryTypes
using LCMGL
using Meshing
using DrakeVisualizer
using BenchmarkTools

In [218]:
include("gjk.jl")

Gjk

In [220]:
s = Simplex(Vec(2., -1), Vec(2.5, 0), Vec(2., 1), Vec(1., 1), Vec(1., -1))
Gjk.gjk(s, 10)

(Gjk.Tagged{FixedSizeArrays.Vec{2,Float64}}[Gjk.Tagged{FixedSizeArrays.Vec{2,Float64}}(Vec(2.0,-1.0),0),Gjk.Tagged{FixedSizeArrays.Vec{2,Float64}}(Vec(1.0,1.0),0),Gjk.Tagged{FixedSizeArrays.Vec{2,Float64}}(Vec(1.0,-1.0),0)],Vec(1.0,0.0))

In [222]:
mesh = DrakeVisualizer.contour_mesh(x -> sum((x - [2, -1.5, 0]).^2) - 1, [-1.5, -1.5, -1.5], [1.5, 1.5, 1.5])
simplex = Gjk.gjk(mesh)[1]
Visualizer(mesh)
LCMGLClient("simplex") do lcmgl
    color(lcmgl, 1, 0, 0, 0.5)
    for tagged in simplex
        sphere(lcmgl, [tagged.point...], 0.02, 20, 20)
    end
    color(lcmgl, 0, 0, 1, 0.5)
    sphere(lcmgl, [0., 0, 0], 0.02, 20, 20)
    switch_buffer(lcmgl)
end

In [227]:
acc = Gjk.AcceleratedMesh(mesh)
simplex, pt_best = Gjk.gjk(acc)
Visualizer(mesh)
LCMGLClient("simplex") do lcmgl
    color(lcmgl, 1, 0, 0, 0.5)
    for tagged in simplex
        sphere(lcmgl, [tagged.point...], 0.02, 20, 20)
    end
    color(lcmgl, 0, 0, 1, 0.5)
    sphere(lcmgl, [0., 0, 0], 0.02, 20, 20)
    switch_buffer(lcmgl)
end

In [224]:
@benchmark Gjk.gjk($mesh)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  5.73 kb
  allocs estimate:  161
  minimum time:     51.33 μs (0.00% GC)
  median time:      53.65 μs (0.00% GC)
  mean time:        58.54 μs (2.46% GC)
  maximum time:     8.41 ms (97.37% GC)

In [225]:
@benchmark Gjk.gjk($acc)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  7.23 kb
  allocs estimate:  193
  minimum time:     55.56 μs (0.00% GC)
  median time:      62.16 μs (0.00% GC)
  mean time:        66.55 μs (2.53% GC)
  maximum time:     6.25 ms (96.55% GC)

In [229]:
@benchmark Gjk.gjk($mesh, $simplex, $pt_best)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     9
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  736.00 bytes
  allocs estimate:  26
  minimum time:     2.36 μs (0.00% GC)
  median time:      2.44 μs (0.00% GC)
  mean time:        2.68 μs (3.91% GC)
  maximum time:     562.17 μs (98.33% GC)

In [228]:
@benchmark Gjk.gjk($acc, $simplex, $pt_best)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     10
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  784.00 bytes
  allocs estimate:  27
  minimum time:     1.50 μs (0.00% GC)
  median time:      1.57 μs (0.00% GC)
  mean time:        1.73 μs (7.62% GC)
  maximum time:     506.83 μs (98.48% GC)